# 2489-2122 Data Curation T3 Final Exam 

The final exam will contain 1 question with subquestions for 70% of the total points. 

<br><br>

## Aircraft Wildlife Strikes

![](https://i.pinimg.com/736x/30/de/ee/30deee2cbc1aec96a1532c4a27962f0c--bird-strike-passenger-aircraft.jpg)

The "wildlifestrikes.csv" dataset contains a record of each reported wildlife strike of military, commercial, or civil aircraft between 1990 and 2015. Each row contains the incident date, aircraft operator, aircraft make and model, engine make and model, airport name and location, species name and quantity, and aircraft damage.
The wildlife strike database was compiled from reports received from airports, airlines, and pilots and published by the Federal Aviation Association. 
<br>

**Question 3.1** (5 points) Import the dataset "wildlifestrikes.csv" and show the ***total number of records***, the ***number of columns***, and how many ***unique species*** the dataset has.

In [50]:
import numpy as np 
import pandas as pd 

database = 'wildlifestrikes.csv'
df = pd.read_csv(database, low_memory=False)

print(df.shape)

columns = df.columns
number_of_columns = len(a)
print('Number of columns :',number_of_columns)

df_species = df['Species ID'].unique()

df_species_d = pd.DataFrame(df_species)
number_of_rows = len(df_species_d)

print('Number of unique species :',number_of_rows)

(174104, 66)
Number of columns : 66
Number of unique species : 719


**Question 3.2** (10 points) 
In the year 2000, what percentage of flights suffered a wild animal strike and had to perform a "PRECAUTIONARY LANDing"? (hint: check the "Flight Impact" column in the dataset). Report the result to two decimal places.

In [36]:
data_2000 = df.loc[df["Incident Year"] == 2000]
total = len(data_2000)
data_2000_landed = data_2000[data_2000["Flight Impact"] == "PRECAUTIONARY LANDING"]

P = len(data_2000_landed) / total
Percentage = "{:.2f}".format(P)
print(Percentage)

0.05


**Question 3.3** (5 points) What was the year with the highest number of fatalities due to a wildlife strike?
(hint: check the column "Fatalities" on the dataset)

In [41]:
df.fillna(0)
data_fatalities = df.groupby('Incident Year')['Fatalities'].count().reset_index()
data_fatalities.max()

Incident Year    2015
Fatalities        553
dtype: int64

**Question 3.4** (10 points) Find the top 5 species that caused engine damage on the aircraft after striking it (remember that there are up to 4 different possible engines on an aircraft). For this exercise, you can consider unknown species.

In [45]:
data_damaged = df[(df['Engine1 Damage'] == 1) | (df['Engine2 Damage'] == 1) | (df['Engine3 Damage'] == 1) | (df['Engine4 Damage'] == 1)]
data_damaged_species_unclean = data_damaged.groupby('Species Name').sum()
data_damaged_species = data_damaged_species_unclean[['Engine1 Damage','Engine2 Damage','Engine3 Damage','Engine4 Damage']]
data_damaged_species['Total_engine_damage'] = data_damaged_species['Engine1 Damage'] + data_damaged_species['Engine2 Damage'] + data_damaged_species['Engine3 Damage'] + data_damaged_species['Engine4 Damage']
data_sorted = data_damaged_species.sort_values(by = ['Total_engine_damage'], ascending=False)
data_sorted.head(5)


C:\Users\Omar\AppData\Local\Temp/ipykernel_19188/1733290355.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_damaged_species['Total_engine_damage'] = data_damaged_species['Engine1 Damage'] + data_damaged_species['Engine2 Damage'] + data_damaged_species['Engine3 Damage'] + data_damaged_species['Engine4 Damage']


,Engine1 Damage,Engine2 Damage,Engine3 Damage,Engine4 Damage,Total_engine_damage
Species Name,,,,,
UNKNOWN MEDIUM BIRD,812,703,67,29,1611
GULL,202,178,9,7,396
UNKNOWN SMALL BIRD,147,106,10,2,265
UNKNOWN LARGE BIRD,114,94,13,1,222
CANADA GOOSE,116,87,9,4,216


**Question 3.5** (10 points) Find the name of the specie that was responsible for the most incidents and indicate the total number of incidents caused by it. The result cannot be an unknown bird.

In [47]:
data_species = df.groupby('Species Name')['Record ID'].count().reset_index()
df_sort = data_species.sort_values(by=['Record ID'], ascending=False)

for index, row in df_sort.iterrows():
    if 'UNKNOWN' in row['Species Name']:
        continue
    else:
        print ('Species Name :', row['Species Name'])
        print ('Number of incidents :', row['Record ID'])

        break;

Species Name : MOURNING DOVE
Number of incidents : 7653


**Question 3.6** (15 points) Show how many incidents occurred every five years (i.e. total number of incidents for each of the following time intervals: (1990, 1995], (1995, 2000], (2000, 2005], (2005, 2010], and (2010,2015]). Note that the time intervals are open on the left and closed on the right.

In [101]:
df["year_bins"] = pd.cut(x = df["Incident Year"], bins = [1990, 1995, 2000, 2005, 2010, 2015], include_lowest=True, labels=["90-95", "95-00", "00-05", "05-10", "10-15"])

data_binned = df.groupby('year_bins')['Record ID'].count().reset_index()
df_sort = data_binned.sort_values(by=['Record ID'], ascending=False)

for index, row in df_sort.iterrows():
        print ('Year Interval :', row['year_bins'])
        print ('Number of incidents :', row['Record ID'])

Year Interval : 10-15
Number of incidents : 57417
Year Interval : 05-10
Number of incidents : 43518
Year Interval : 00-05
Number of incidents : 34073
Year Interval : 95-00
Number of incidents : 22733
Year Interval : 90-95
Number of incidents : 16363


**Question 3.7** (15 points) Consider the five-time intervals: (1990, 1995], (1995, 2000], (2000, 2005], (2005, 2010], and (2010,2015]. Use a pivot_table to find out, ***on each time interval***, which type of aircraft ("Aircraft") got most frequently damaged ("Aircraft Damage") after a wildlife strike. (hint: First, do the pivot table with the total sum of the damage per each type of aircraft suffered at each time interval. After that, obtain the aircraft type with the highest value for each time interval.)

In [103]:
data_90_95 = df.loc[df["year_bins"] == "90-95"]
data_95_00 = df.loc[df["year_bins"] == "95-00"]
data_00_05 = df.loc[df["year_bins"] == "00-05"]
data_05_10 = df.loc[df["year_bins"] == "05-10"]
data_10_15 = df.loc[df["year_bins"] == "10-15"]

data_90_95.pivot_table(["Aircraft", "Aircraft Damage"])

,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,Aircraft Make,Aircraft Model,...,Fuselage Damage,Landing Gear Strike,Landing Gear Damage,Tail Strike,Tail Damage,Lights Strike,Lights Damage,Other Strike,Other Damage,year_bins
0,127128,1990,1,1,DAL,DELTA AIR LINES,B-757-200,A,148,26,...,0,0,0,1,1,0,0,0,0,90-95
1,129779,1990,1,1,HAL,HAWAIIAN AIR,DC-9,A,583,90,...,0,0,0,0,0,0,0,1,0,90-95
2,129780,1990,1,2,UNK,UNKNOWN,UNKNOWN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,90-95
3,2258,1990,1,3,MIL,MILITARY,A-10A,A,345,NaN,...,0,0,0,0,0,0,0,0,0,90-95
4,2257,1990,1,3,MIL,MILITARY,F-16,A,561,NaN,...,0,0,0,0,0,0,0,0,0,90-95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16358,126366,1995,12,31,UAL,UNITED AIRLINES,B-767-200,A,148,29,...,0,0,0,0,0,0,0,0,0,90-95
16359,116435,1995,12,31,BUS,BUSINESS,B-737-300,A,148,24,...,0,0,0,0,0,0,0,0,0,90-95
16360,116420,1995,12,31,USA,1US AIRWAYS,DC-9-30,A,583,21,...,0,0,0,0,0,0,0,0,0,90-95
16361,116478,1995,12,31,EGF,AMERICAN EAGLE AIRLINES,SAAB-340,A,778,5,...,0,0,0,0,0,0,0,0,0,90-95
